In [4]:
using DynamicalBilliards

In [5]:
r=1 #radius of semi-circles

1

In [6]:
bd=Obstacle{Float64}[]

Obstacle{Float64}[]

In [7]:
s1=Semicircle([-1,0],r,[2,0])
s2=Semicircle([0,1],r,[0,-2])
s3=Semicircle([1,0],r,[-2,0])
s4=Semicircle([0,-1],r,[0,2])

Semicircle {Float64}
center: [0.0, -1.0]
radius: 1.0
facedir: [0.0, 1.0]

In [8]:
push!(bd,s1)
push!(bd,s2)
push!(bd,s3)
push!(bd,s4)

4-element Array{Obstacle{Float64},1}:
 Semicircle {Float64}
center: [-1.0, 0.0]
radius: 1.0
facedir: [1.0, 0.0]
 Semicircle {Float64}
center: [0.0, 1.0]
radius: 1.0
facedir: [0.0, -1.0]
 Semicircle {Float64}
center: [1.0, 0.0]
radius: 1.0
facedir: [-1.0, 0.0]
 Semicircle {Float64}
center: [0.0, -1.0]
radius: 1.0
facedir: [0.0, 1.0]

In [9]:
bill=Billiard(bd)

Billiard{Float64} with 4 obstacles:
  Semicircle
  Semicircle
  Semicircle
  Semicircle

In [ ]:
using Plots

In [ ]:
pyplot()

In [20]:
pi

π = 3.1415926535897...

In [23]:
p1=Particle(0,0,(3/5)pi)
p2=randominside(bill)

Particle{Float64}
position: [1.7678827767662346, -0.1541831687305244]
velocity: [0.9996254931721396, 0.027365551343920654]

In [ ]:
import DynamicalBilliards: increment_counter

In [ ]:
import DynamicalBilliards: isperiodic

In [ ]:
function evolve!(p::AbstractParticle{T}, bd::Billiard{T}, t, raysplitters = nothing;
    warning = false) where {T<:AbstractFloat}

    if t ≤ 0
        throw(ArgumentError("`evolve!()` cannot evolve backwards in time."))
    end
    if ispinned(p, bd)
        push!(rpos, p.pos)
        push!(rvel, p.vel)
        push!(rt, Inf)
        return (rt, rpos, rvel, p.ω)
    end

    ismagnetic = p isa MagneticParticle
    isray = !isa(raysplitters, Nothing)
    isray && acceptable_raysplitter(raysplitters, bd)
    raysidx = raysplit_indices(bd, raysplitters)
    ismagnetic && isray && (omegas = [p.ω])

    rt = T[0.0]; rpos = [p.pos]; rvel = [p.vel]
    count = zero(t)
    flight = zero(T)
    if typeof(t) == Int
        for zzz in (rt, rpos, rvel)
            sizehint!(zzz, t)
        end
    end

    while count < t

        i, tmin, pos, vel = bounce!(p, bd, raysidx, raysplitters)
        flight += tmin

        if isperiodic(i, bd)
            continue
        else
            push!(rpos, pos + p.current_cell)
            push!(rvel, vel)
            push!(rt, flight)
            ismagnetic && isray && push!(omegas, p.ω)
            # set counter
            count += increment_counter(t, flight)
            flight = zero(T)
        end
        
        if pos==(-r,r)||pos==(r,r)||pos==(r,-r)||pos==(-r,-r)
            break;
        end
    end  #time, or collision number, loop

    # Return stuff
    if ismagnetic && isray
        return (rt, rpos, rvel, omegas)
    elseif ismagnetic
        return (rt, rpos, rvel, p.ω)
    else
        return (rt, rpos, rvel)
    end
end

In [ ]:
history1=evolve!(p1,bill,20)
history2=evolve!(p2,bill,20)

In [ ]:
PyPlot.plot(bill)
for i=1:lastindex(history1[2])
    l1=[history1[2][i][1],history1[2][i][2]]
    l2=[history2[2][i][1],history2[2][i][2]]
    PyPlot.scatter(l1[1],l1[2])
    PyPlot.scatter(l2[1],l2[2])
    if i==1
        continue
    end
    l01=history1[2][i-1][1],history1[2][i-1][2]
    l02=history2[2][i-1][1],history2[2][i-1][2]
    PyPlot.plot((l1[1],l01[1]),(l1[2],l01[2]), color=:blue)
    PyPlot.plot((l2[1],l02[1]),(l2[2],l02[2]), color=:red)
end